## StarWat/SmartSeeds segmentation predictions without mask use ##

We will use in this notebook trained segmentation models to perform segmentation on CARE projected images. You can use other 2D images not generated thanks to CARE but the prediction results will be less good. 

For this notebook, we won't use binary mask to help the segmentation because there is no background in the image or because we are only interested in some clone cells. 

In [6]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import os
import cv2

#To run the prediction on the GPU, else comment out this line to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import csbdeep
from csbdeep.utils.tf import limit_gpu_memory
from skimage.filters import threshold_otsu
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'retina'")
from IPython.display import clear_output
import glob
import cv2
from tifffile import imread
import tqdm
from csbdeep.utils import Path, normalize
from stardist import random_label_cmap # ,_draw_polygons,dist_to_coord
from stardist.models import StarDist2D
from TerminatorUtils.helpers import normalizeFloat, OtsuThreshold2D, save_8bit_tiff_imagej_compatible, save_16bit_tiff_imagej_compatible,Integer_to_border,SeedStarDistWatershed
from TerminatorUtils.helpers import Prob_to_Binary, zero_pad, WatershedwithoutMask
from TerminatorUtils.helpers import save_tiff_imagej_compatible
from csbdeep.models import Config, CARE
np.random.seed(6)
lbl_cmap = random_label_cmap()

from skimage.morphology import remove_small_objects
from skimage.morphology import skeletonize
import time 

import time

#from TerminatorUtils.helpers import limit_gpu_memory_actualised
#limit_gpu_memory_actualised(0.25)


All the images we want to segment must be contained in the same folder and must be .tif. We will give the path to the folder through `basedir`.

- `basedir_StardistWater_NoMask` will be the path were segmented images will be saved,
- `basedir_StardistWater_NoMask_RTGPipeline` will be the path were segmented images put in the good format for Guirao Pipeline will be saved


`Model_Dir` is the path where all the Segmentation trained models are stored.

In [64]:
basedir = '/run/user/1000/gvfs/smb-share:server=isiserver.curie.net,share=u934/equipe_bellaiche/l_sancere/Yo_To_Lucas/171020_tosend 2/trainning/AllMovies'

basedir_StardistWater_NoMask = basedir + '/StarWat_NoMask/'
basedir_StardistWater_NoMask_RTGPipeline = basedir + '/StarWat_NoMask_RTGPipeline/'

Model_Dir = '/run/media/sancere/DATA/Lucas_Model_to_use/Segmentation/'    

Here we select the model we will use for the prediction and we load it.

In [65]:
StardistModelName = 'DrosophilaSegmentationSmartSeedsMatureMacroElongatedLargePatchBoris_29Jan2021_Grid11_PreviousHP'

modelstar = StarDist2D(config = None, name = StardistModelName, basedir = Model_Dir)

Loading network weights from 'weights_best.h5'.
Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.


Then we create the `basedir_StardistWater_NoMask` folders were the predictions will be stored if the folder doesn't exist yet. We then list all the files with the given file extension (* *choose extenstion*) in the `basedir` folder.

In [66]:
Path(basedir_StardistWater_NoMask).mkdir(exist_ok = True)
Raw_path = os.path.join(basedir, '*.tif') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw = glob.glob(Raw_path)
filesRaw.sort

<function list.sort>

We finally apply the predictions on each  images found in the folder. 

The 2d line is to avoid to make a segmentation prediction on an image that was already processed by the network before.

In [67]:
for fname in filesRaw:
 
    if os.path.exists((basedir_StardistWater_NoMask + 'StarWat_NoMask_' + os.path.basename(os.path.splitext(fname)[0]))) == False :
        
            # Name = os.path.basename(os.path.splitext(fname)[0])
            x = imread(fname)
            y = normalizeFloat(x,1,99.8, axis = axis_norm)
            originalX = x.shape[0]
            originalY = x.shape[1]
        
            #zero pad (Never do 8bit conversion when applying prediction on an image, only float32)
            y = zero_pad(y, 32, 32)
            print('Processing file', fname)              
           
            #Stardist, label image, details, probability map, distance map 
            segmentationImage, details = modelstar.predict_instances(y)
            prob, dist = modelstar.predict(y)
            grid = modelstar.config.grid
            prob = cv2.resize(prob,  dsize=(prob.shape[1] * grid[1],prob.shape[0] * grid[0]))
            
            
            #Seeds from Stardist + Whatershed, segmentation on probability map 
            Watershed, markers = WatershedwithoutMask(prob, segmentationImage.astype('uint16'), modelstar.config.grid)   
                                                                   
            
            #Convert Integer image to binary
            Binary_Watershed = Integer_to_border(Watershed[:originalX, :originalY])
           
            
            #Skeletonization and deletion of non closed cells. Has to be done AFTER logical and operation 
            z = Binary_Watershed 
            z = skeletonize(z) 
            z = np.float32(z)
            z2 = z.copy()
            mask = np.zeros((np.array(z.shape)+2), np.uint8)
            cv2.floodFill(z, mask, (0,0), (255))
            z = cv2.erode(z, np.ones((3,3)))
            z = cv2.bitwise_not(z)
            z = cv2.bitwise_and(z,z2)
            StardistWater_NoMask = z 
    
            #Save different method segmentations
            save_8bit_tiff_imagej_compatible((basedir_StardistWater_NoMask + 'StarWat_NoMask_' + os.path.basename(os.path.splitext(fname)[0]) ) , StardistWater_NoMask, axes)



Processing file /run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/Movie1_CyanMarkerOnly_0001.tif
Processing file /run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/Movie1_CyanMarkerOnly_0002.tif
Processing file /run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/Movie1_CyanMarkerOnly_0003.tif
Processing file /run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/Movie1_CyanMarkerOnly_0004.tif
Processing file /run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/Movie1_CyanMarkerOnly_0005.tif
Processing file /run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/Movie1_CyanMarkerOnly_0006.tif
Processing file /run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/Movie1_CyanMarkerOnly_0007.tif
Processing file /run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOn

In the last cell we convert the segmented prediction in png and change the names for them to be used directly by Guirao Pipeline. We will end up with 2 folders of predictions in different formats.

In [68]:
import png

Path(basedir_StardistWater_NoMask_RTGPipeline).mkdir(exist_ok = True)

Raw_path_2 = os.path.join(basedir_StardistWater_NoMask, '*') #tif or TIF be careful
axis_norm = (0,1)   # normalize channels independently
axes = 'XY'
filesRaw_2 = glob.glob(Raw_path_2)

    
for fname in filesRaw_2:
    z = imread(fname)
    z = 255 * z
    z = cv2.bitwise_not(z)
    name = str(os.path.basename(os.path.splitext(fname)[0])).rstrip(".tif")
    png.from_array(z, mode="L").save(basedir_StardistWater_NoMask_RTGPipeline + name.replace("StarWat_NoMask_",'seg_') + '.png') 

/run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/StarWat_NoMask/StarWat_NoMask_Movie1_CyanMarkerOnly_0001
StarWat_NoMask_Movie1_CyanMarkerOnly_0001
/run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/StarWat_NoMask/StarWat_NoMask_Movie1_CyanMarkerOnly_0002
StarWat_NoMask_Movie1_CyanMarkerOnly_0002
/run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/StarWat_NoMask/StarWat_NoMask_Movie1_CyanMarkerOnly_0003
StarWat_NoMask_Movie1_CyanMarkerOnly_0003
/run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/StarWat_NoMask/StarWat_NoMask_Movie1_CyanMarkerOnly_0004
StarWat_NoMask_Movie1_CyanMarkerOnly_0004
/run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/StarWat_NoMask/StarWat_NoMask_Movie1_CyanMarkerOnly_0005
StarWat_NoMask_Movie1_CyanMarkerOnly_0005
/run/media/sancere/DATA1/Segmentation_Challenge/Movie1_CyanMarkerOnly/MedianFilter3/S